In [ ]:
!pip install transformers==4.15.0 sentencepiece
!pip install datasets==1.17.0 
!pip install pythainlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.4 MB 7.3 MB/s 
     |████████████████████████████████| 1.2 MB 52.7 MB/s 
     |████████████████████████████████| 880 kB 52.7 MB/s 
     |████████████████████████████████| 3.3 MB 31.0 MB/s 
     |████████████████████████████████| 596 kB 41.7 MB/s 
     |████████████████████████████████| 86 kB 5.2 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=558228c6c9f4a66e4bcade9c52d69cf7818bc125978a1f24814b575a00474348
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |█████

In [ ]:
from transformers import AutoModelForMaskedLM, pipeline
from transformers import AutoTokenizer, BertForTokenClassification
import pandas as pd
import torch
import pickle
from tqdm import tqdm
from datasets import load_metric
from pythainlp.benchmarks import word_tokenization

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [ ]:
# change the input directory to your own preferences
tokenizer = pickle.load(open('drive/MyDrive/AIBuilders/tokenizer.pkl', 'rb'))

In [ ]:
class BertModel(torch.nn.Module):
    def __init__(self):
        super(BertModel, self).__init__()
        self.bert = BertForTokenClassification.from_pretrained('airesearch/wangchanberta-base-att-spm-uncased', num_labels=2)
        self.bert.resize_token_embeddings(len(tokenizer))

    def forward(self, input_id, mask, label):
        output = self.bert(input_ids=input_id, attention_mask=mask, labels=label, return_dict=False)
        return output

# change the input directory to your own preferences
FILE = "drive/MyDrive/AIBuilders/tagging.pth"
tagging_model = BertModel()
tagging_model.load_state_dict(torch.load(FILE, map_location=torch.device('cpu')))
tagging_model.eval()

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

You are using a model of type camembert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Downloading:   0%|          | 0.00/404M [00:00<?, ?B/s]

Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing BertForTokenClassification: ['roberta.encoder.layer.4.attention.output.LayerNorm.weight', 'roberta.encoder.layer.5.attention.output.dense.bias', 'roberta.encoder.layer.3.attention.self.query.bias', 'roberta.encoder.layer.2.attention.output.LayerNorm.bias', 'roberta.encoder.layer.10.attention.output.LayerNorm.bias', 'roberta.encoder.layer.11.attention.self.value.weight', 'roberta.encoder.layer.11.attention.output.dense.bias', 'roberta.encoder.layer.1.output.dense.bias', 'roberta.encoder.layer.7.attention.self.value.bias', 'roberta.embeddings.position_embeddings.weight', 'roberta.encoder.layer.1.attention.self.query.weight', 'roberta.encoder.layer.8.output.LayerNorm.bias', 'roberta.encoder.layer.5.intermediate.dense.weight', 'roberta.encoder.layer.4.attention.output.LayerNorm.bias', 'roberta.encoder.layer.0.attention.self.key.weight', 'roberta.encoder.layer.5.output.Layer

BertModel(
  (bert): BertForTokenClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(33660, 768)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (Lay

In [ ]:
ids_to_labels = {0: 'f', 1: 'i'}

def evaluate_one_text(model, sentence, mask, labels):
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:
        model = model.cuda()

    input_id = torch.Tensor([sentence]).type(torch.int64)
    label_ids = []
    for i in sentence:
      if i == 1 or i == 5 or i == 6:
        label_ids.append(-100)
      else:
        label_ids.append(2)
    label_ids = torch.Tensor([label_ids]).type(torch.int64)
    mask = torch.Tensor([mask]).type(torch.int64)

    logits = tagging_model(input_id, mask, None)
    logits_clean = logits[0][label_ids != -100]

    predictions = logits_clean.argmax(dim=1).tolist()
    prediction_label = [ids_to_labels[i] for i in predictions]
    return prediction_label

In [ ]:
# change the input directory to your own preferences
FILE = "drive/MyDrive/AIBuilders/mlm.pth"

model_checkpoint = "airesearch/wangchanberta-base-att-spm-uncased"
mlm_model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)
mlm_model.resize_token_embeddings(len(tokenizer))
mlm_model.load_state_dict(torch.load(FILE, map_location=torch.device('cpu')))
mlm_model = mlm_model.to(device)
mlm_model.eval()

CamembertForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(33660, 768)
      (position_embeddings): Embedding(512, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps

In [ ]:
# change the input directory to your own preferences
ds_mlm = pickle.load(open('drive/MyDrive/AIBuilders/mlm_ds.pkl', 'rb'))
ds_tag = pickle.load(open('drive/MyDrive/AIBuilders/tagging_ds.pkl', 'rb'))

In [ ]:
ds_tag

,text,labels
0,"{'input_ids': [[tensor(5), tensor(10), tensor(...","[tensor(0), tensor(0), tensor(0), tensor(0), t..."
1,"{'input_ids': [[tensor(5), tensor(10), tensor(...","[tensor(0), tensor(0), tensor(0), tensor(0), t..."
2,"{'input_ids': [[tensor(5), tensor(984), tensor...","[tensor(0), tensor(0), tensor(0), tensor(0), t..."
3,"{'input_ids': [[tensor(5), tensor(10), tensor(...","[tensor(0), tensor(0), tensor(0), tensor(0), t..."
4,"{'input_ids': [[tensor(5), tensor(10), tensor(...","[tensor(0), tensor(0), tensor(0), tensor(0), t..."
...,...,...
42885,"{'input_ids': [[tensor(5), tensor(10), tensor(...","[tensor(0), tensor(0), tensor(0), tensor(0), t..."
42886,"{'input_ids': [[tensor(5), tensor(10), tensor(...","[tensor(0), tensor(0), tensor(0), tensor(0), t..."
42887,"{'input_ids': [[tensor(5), tensor(10), tensor(...","[tensor(0), tensor(0), tensor(1), tensor(1), t..."
42888,"{'input_ids': [[tensor(5), tensor(13276), tens...","[tensor(0), tensor(0), tensor(0), tensor(0), t..."


In [ ]:
def ids_to_tokens(tokenized_text):
  a = tokenizer.convert_ids_to_tokens(tokenized_text)
  a.remove("<s>")
  a.remove("</s>")
  if a[0] == '▁':
    a.pop(0)
  return a

In [ ]:
NUM_SAMPLE = 1000
bleu = load_metric("bleu")
totalacc = 0
bleu_higher = 0
bleu_lower = 0
bleu_equal = 0
total_f1 = 0

for sent_id in tqdm(range(NUM_SAMPLE)):
  chng = []
  text = ds_tag.iloc[sent_id]['text']['input_ids'].squeeze(0).tolist()
  mask = ds_mlm.iloc[sent_id]['attention_mask']
  labels = ds_mlm.iloc[sent_id]['labels']
  original = [ids_to_tokens(text)]
  references = [[ids_to_tokens(labels)]]
  # print(f"TEXT: {original}")
  

  i_f = evaluate_one_text(tagging_model, text, mask, labels)
  predicted_id = text.copy()
  original_id = text.copy()
  i_f_len = len(i_f)
  for j in range(i_f_len):
    if(i_f[j] == 'i'):
      ph = predicted_id[j+1]

      predicted_id[j+1] = 25004
      mlm_input = {'input_ids': torch.Tensor([predicted_id]).type(torch.int64).to(device), 'attention_mask': torch.Tensor([mask]).type(torch.int64).to(device)}
      token_logits = mlm_model(**mlm_input).logits
      mask_token_index = torch.where(mlm_input["input_ids"] == tokenizer.mask_token_id)[1]
      mask_token_logits = token_logits[0, mask_token_index, :]
      top_5_tokens = torch.topk(mask_token_logits, 3, dim=1).indices[0].tolist()
      # print(f"{tokenizer.convert_ids_to_tokens(ph)} => {tokenizer.convert_ids_to_tokens(top_5_tokens[0])}")
      chng.append((j, top_5_tokens[0]))

      predicted_id[j+1] = ph

  for x,y in chng:
    predicted_id[x+1] = y

  numer = 0
  denom = 0
  TP = 0
  FP = 0
  TN = 0
  FN = 0
  labels_len = len(labels)
  for i in range(labels_len):
    if not predicted_id[i] == original_id[i]: #change
      denom += 1
      if predicted_id[i] == labels[i]:
        numer += 1
        TP += 1
      elif not predicted_id[i] == labels[i]:
        FP += 1
    elif predicted_id[i] == original_id[i]: #no change
      if predicted_id[i] == labels[i]:
        TN += 1
      elif not predicted_id[i] == labels[i]:
        FN += 1
  if denom == 0:
    acc = 0
  else:
    acc = float(numer)/float(denom)
  totalacc += acc

  # print(f"TP:{TP}   TN:{TN}   FP:{FP}   FN:{FN}")
  precision = float(TP) / float(TP+FP) if TP+FP > 0 else 0
  recall = float(TP) / float(TP+FN) if TP+FN > 0 else 0
  f1 = float(2*precision*recall) / float(precision + recall) if precision+recall > 0 else 0
  total_f1 += f1
  

  ans = tokenizer.convert_ids_to_tokens(predicted_id)
  ans.remove('<s>')
  ans.remove('</s>')
  if ans[0] == '▁':
    ans.pop(0)
  predictions = [ans]  
  ans = ''.join(ans)
  ans = ans.replace("▁", " ")

  bleu_original = bleu.compute(predictions=original, references=references)
  bleu_prediction = bleu.compute(predictions=predictions, references=references)
  
  if bleu_prediction['bleu'] > bleu_original['bleu']:
    bleu_higher += 1
  elif bleu_prediction['bleu'] < bleu_original['bleu']:
    bleu_lower += 1
  elif bleu_prediction['bleu'] == bleu_original['bleu']:
    bleu_equal += 1

  # print(ans)
  # print(f"ACC: {acc}")
  # print(f"f1: {f1}")
  # print(f"ORIGINAL : {bleu_original}")
  # print(f"PREDICTED: {bleu_prediction}")
  # print("--------------------------------------------")
print(f"AVG ACC: {float(totalacc/NUM_SAMPLE)}")
print(f"AVG f1: {float(total_f1/NUM_SAMPLE)}")
print(f"# HIGHER BLEU PREDICTION: {bleu_higher}")
print(f"# LOWER BLEU PREDICTION: {bleu_lower}")
print(f"# EQUAL BLEU PREDICTION: {bleu_equal}")

 31%|███▏      | 313/1000 [46:44<1:42:51,  8.98s/it]